<a href="https://colab.research.google.com/github/advik-7/NLP_projects/blob/main/untrained_language_translator_encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("hey")

hey


In [4]:
import zipfile
with zipfile.ZipFile("/content/archive (5).zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split


In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("/content/data/eng_-french.csv")

In [8]:
df

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !
...,...,...
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [9]:
df.isnull().sum().sum()

0

In [10]:

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(df['English words/sentences'])
eng_sequences = eng_tokenizer.texts_to_sequences(df['English words/sentences'])
eng_input = pad_sequences(eng_sequences, padding='post')

In [12]:
fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(df['French words/sentences'].values)
fr_sequences = fr_tokenizer.texts_to_sequences(df['French words/sentences'].values)
fr_input = pad_sequences(fr_sequences, padding='post')


In [13]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1





In [14]:

X_train, X_val, y_train, y_val = train_test_split(eng_input, fr_input, test_size=0.2)

In [15]:
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [16]:

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=ENC_EMB_DIM)(encoder_inputs)
encoder_dropout = Dropout(ENC_DROPOUT)(encoder_embedding)

encoder_lstm, state_h, state_c = LSTM(HID_DIM, return_sequences=True, return_state=True)(encoder_dropout)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=fr_vocab_size, output_dim=DEC_EMB_DIM)(decoder_inputs)
decoder_dropout = Dropout(DEC_DROPOUT)(decoder_embedding)

decoder_lstm, _, _ = LSTM(HID_DIM, return_sequences=True, return_state=True)(decoder_dropout, initial_state=encoder_states)
decoder_outputs = Dense(fr_vocab_size, activation='softmax')(decoder_lstm)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [18]:

model.fit([X_train, y_train[:, :-1]], y_train[:, 1:], epochs=1, batch_size=32, validation_data=([X_val, y_val[:, :-1]], y_val[:, 1:]))


 127/4391 ━━━━━━━━━━━━━━━━━━━━ 9:26:00 8s/step - loss: 3.0009

KeyboardInterrupt: 